In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "EMBEDDIA/sloberta"

In [3]:
from datasets import load_dataset

dataset = load_dataset("anzeo/Slovene_SuperGLUE_CB")

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

C:\Users\anzeo\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
from transformers import set_seed

set_seed(42)

id2label = {0: "entailment", 1: "neutral", 2: "contradiction"}
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
model.config.use_cache = False

C:\Users\anzeo\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\anzeo\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/sloberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
Y

In [6]:
CONTEXT_COL = "premise"
HYPOTHESIS_COL = "hypothesis"

def preprocess_function(examples):
    """
    The preprocessing function prepares examples for processing by the model.
    It concatenates premise and hypothesis for each example to form a single input string.
    """
    inputs = [f"{premise} {hypothesis}" for premise, hypothesis in zip(examples[CONTEXT_COL], examples[HYPOTHESIS_COL])]
    tokenized_examples = tokenizer(inputs, truncation=True)
    if "label" in examples:
        tokenized_examples["labels"] = [label2id[label] for label in examples["label"]]
    return tokenized_examples

In [7]:
tokenized_dataset = dataset.map(preprocess_function,
                                remove_columns=['idx', 'premise', 'hypothesis', 'label'], batched=True)

In [8]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 110
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 22
    })
})

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
import numpy as np

from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [11]:
from peft import LoraConfig, get_peft_model, TaskType

lora_alpha = 32
lora_rank_dropout = 0.1
lora_module_dropout = 0.0
lora_r = 16

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=lora_r,
    lora_alpha=lora_alpha,
    bias="none",
    target_modules=['query', 'value'],
    base_model_name_or_path=model_name
)

In [12]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,182,723 || all params: 111,806,982 || trainable%: 1.0578


In [13]:
from transformers import TrainingArguments

new_model_name = "lora_fine_tuned_cb_sloberta"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [15]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=400, training_loss=0.7458982467651367, metrics={'train_runtime': 35.814, 'train_samples_per_second': 89.351, 'train_steps_per_second': 11.169, 'total_flos': 186646655447928.0, 'train_loss': 0.7458982467651367, 'epoch': 28.571428571428573})

In [16]:
trainer.evaluate()

{'eval_loss': 1.4606362581253052,
 'eval_accuracy': 0.3181818181818182,
 'eval_f1': 0.1536050156739812,
 'eval_runtime': 0.1725,
 'eval_samples_per_second': 127.538,
 'eval_steps_per_second': 17.391,
 'epoch': 28.571428571428573}

In [17]:
trainer.push_to_hub(new_model_name)

C:\Users\anzeo\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/4.74M [00:00<?, ?B/s]

events.out.tfevents.1716375259.DESKTOP-22QTFDR.13792.5:   0%|          | 0.00/457 [00:00<?, ?B/s]

events.out.tfevents.1716375223.DESKTOP-22QTFDR.13792.4:   0%|          | 0.00/9.90k [00:00<?, ?B/s]

events.out.tfevents.1716375289.DESKTOP-22QTFDR.13792.6:   0%|          | 0.00/9.90k [00:00<?, ?B/s]

...

events.out.tfevents.1716375511.DESKTOP-22QTFDR.13792.10:   0%|          | 0.00/457 [00:00<?, ?B/s]

events.out.tfevents.1716375558.DESKTOP-22QTFDR.16212.0:   0%|          | 0.00/9.90k [00:00<?, ?B/s]

events.out.tfevents.1716375594.DESKTOP-22QTFDR.16212.1:   0%|          | 0.00/457 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/anzeo/lora_fine_tuned_cb_sloberta/commit/8b5fc585f6784994da5da5362be7ed98f5f70b4f', commit_message='lora_fine_tuned_cb_sloberta', commit_description='', oid='8b5fc585f6784994da5da5362be7ed98f5f70b4f', pr_url=None, pr_revision=None, pr_num=None)

In [18]:
# Example
hypothesis ="Valence je pomagal"
premise = "Valence praznoglavi, Valence krepostni kreten. Zakaj si ga tip raje ni zataknil v ustrezen del lastne titanske anatomije? Je morda mislil, da mi pomaga?"

In [19]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSequenceClassification
import torch

# We need to set the seed, otherwise some weights of the model are initialized differently every time, and consequently the result can be different each time as well
set_seed(42)

adapter_name = "anzeo/" + new_model_name

config = PeftConfig.from_pretrained(adapter_name)
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
model = PeftModel.from_pretrained(base_model, adapter_name)

tokenizer = AutoTokenizer.from_pretrained(adapter_name)

adapter_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

C:\Users\anzeo\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anzeo\.cache\huggingface\hub\models--anzeo--lora_fine_tuned_cb_sloberta. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of CamembertForSequenceClassification were not initialized from

adapter_model.safetensors:   0%|          | 0.00/4.74M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/800k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

In [20]:
inputs = tokenizer(f"{premise} {hypothesis}", return_tensors="pt")
label = torch.tensor([1]).unsqueeze(0)  # Batch size 1

outputs = model(**inputs, labels=label)
logits = outputs.logits

In [21]:
# Print prediction
logits.argmax().item()

0